In [3]:
# 1. Importando as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, desc

In [5]:
# 2. Criando a SparkSession apontando para o container do master node (7077)
spark = SparkSession.builder \
    .appName("Spark Demo") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [6]:
#Exemplo 1
# Criando um dataframe que vai de 0 a 999 e retorna quantos números são pares

df = spark.range(1000).toDF("number")


In [8]:
#Criando transformação para filtrar numéros impares
# Note que é apenas um plano de execução. Nenhum resultado é obtido

df_pair = df.where("number % 2 = 0")


In [10]:
#Coloca em operação o plano de execução com a ação count()

#df_pair.count()
df_pair.explain()

== Physical Plan ==
*(1) Project [id#0L AS number#2L]
+- *(1) Filter ((id#0L % 2) = 0)
   +- *(1) Range (0, 1000, step=1, splits=2)




In [12]:
#Exemplo registro de voos Brasil

df_br_flights = spark.read.csv("datasets/br_flights.csv", header=True, inferSchema=True)

df_br_flights.printSchema()

root
 |-- Voos: string (nullable = true)
 |-- Companhia.Aerea: string (nullable = true)
 |-- Codigo.Tipo.Linha: string (nullable = true)
 |-- Partida.Prevista: timestamp (nullable = true)
 |-- Partida.Real: string (nullable = true)
 |-- Chegada.Prevista: timestamp (nullable = true)
 |-- Chegada.Real: string (nullable = true)
 |-- Situacao.Voo: string (nullable = true)
 |-- Codigo.Justificativa: string (nullable = true)
 |-- Aeroporto.Origem: string (nullable = true)
 |-- Cidade.Origem: string (nullable = true)
 |-- Estado.Origem: string (nullable = true)
 |-- Pais.Origem: string (nullable = true)
 |-- Aeroporto.Destino: string (nullable = true)
 |-- Cidade.Destino: string (nullable = true)
 |-- Estado.Destino: string (nullable = true)
 |-- Pais.Destino: string (nullable = true)
 |-- LongDest: double (nullable = true)
 |-- LatDest: double (nullable = true)
 |-- LongOrig: double (nullable = true)
 |-- LatOrig: double (nullable = true)



In [47]:
#Predicate Pushdown
#Ver aba stages e SQL no Spark UI

print("SEM PREDICATE PUSHDOWN")
df_br_flights_parquet = spark.read.parquet("datasets/br_flights.parquet")
df_br_flights_parquet.count()

print("COM PREDICATE PUSHDOWN")
df_partitioned = spark.read.parquet("datasets/br_flights_partitioned")
df_partitioned.filter(df_partitioned["Cidade_Origem"] == "Porto Alegre").count()



COM PREDICATE PUSHDOWN


85470

In [43]:
# 11. Encerrando a SparkSession
spark.stop()